In [18]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.catboost
from catboost import CatBoostRegressor
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional,Dict,Tuple
from pathlib import Path
from enefit_challenge.utils.dataset import load_enefit_training_data

In [7]:
df_train = load_enefit_training_data()
df_train.describe()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,...,week_cosine,month,month_sine,month_cosine,target_2_days_ago,eic_count,installed_capacity,euros_per_mwh,lowest_price_per_mwh,highest_price_per_mwh
count,2.017824e+06,2.017824e+06,2.017824e+06,2.017824e+06,2017824.0,2017824,2.017824e+06,2.017824e+06,2.017824e+06,2.017824e+06,...,2.017824e+06,2.017824e+06,2.017824e+06,2.017824e+06,2.008656e+06,2.009184e+06,2.009184e+06,2.014628e+06,2.014896e+06,2.014896e+06
mean,7.297039e+00,5.368268e-01,1.898927e+00,2.748556e+02,0.5,2022-07-20 08:30:42.518673152,3.218754e+02,1.009178e+06,3.304540e+01,2.022057e+03,...,1.239993e-01,6.430605e+00,3.706141e-03,1.254672e-01,2.747798e+02,7.342647e+01,1.452349e+03,1.574607e+02,9.546883e+01,1.083110e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2021-09-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,2.021000e+03,...,-1.000000e+00,1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,5.000000e+00,6.000000e+00,-1.006000e+01,2.810000e+01,3.400000e+01
25%,3.000000e+00,0.000000e+00,1.000000e+00,3.780000e-01,0.0,2022-02-14 04:00:00,1.660000e+02,5.045818e+05,1.600000e+01,2.022000e+03,...,-4.647232e-01,3.000000e+00,-8.660254e-01,-5.000000e-01,3.740000e-01,1.400000e+01,3.237000e+02,8.529000e+01,6.000000e+01,6.767000e+01
50%,7.000000e+00,1.000000e+00,2.000000e+00,3.113300e+01,0.5,2022-07-21 09:00:00,3.230000e+02,1.009172e+06,3.300000e+01,2.022000e+03,...,2.393157e-01,6.000000e+00,0.000000e+00,6.123234e-17,3.115000e+01,3.200000e+01,6.470800e+02,1.287100e+02,8.590000e+01,9.400000e+01
75%,1.100000e+01,1.000000e+00,3.000000e+00,1.802062e+02,1.0,2022-12-24 14:00:00,4.790000e+02,1.513763e+06,5.000000e+01,2.022000e+03,...,7.485107e-01,1.000000e+01,8.660254e-01,8.660254e-01,1.801290e+02,7.000000e+01,1.567150e+03,1.999700e+02,1.097400e+02,1.330000e+02
max,1.500000e+01,1.000000e+00,3.000000e+00,1.548027e+04,1.0,2023-05-31 23:00:00,6.370000e+02,2.018351e+06,6.800000e+01,2.023000e+03,...,1.000000e+00,1.200000e+01,1.000000e+00,1.000000e+00,1.548027e+04,1.517000e+03,1.931431e+04,4.000000e+03,2.500000e+02,3.050000e+02
std,4.780994e+00,4.986421e-01,1.081766e+00,9.095024e+02,0.5,NaN,1.826310e+02,5.826378e+05,1.959062e+01,6.452196e-01,...,6.650222e-01,3.664933e+00,7.402501e-01,6.605109e-01,9.094796e+02,1.441306e+02,2.423222e+03,1.213226e+02,4.758257e+01,5.476264e+01


In [8]:
df_train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,week_cosine,month,month_sine,month_cosine,target_2_days_ago,eic_count,installed_capacity,euros_per_mwh,lowest_price_per_mwh,highest_price_per_mwh
0,0,0,1,0.713,0,2021-09-01,0,0,0,2021-09-01,...,-0.568065,9,-0.866025,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
1,11,0,2,7.620,1,2021-09-01,0,89,44,2021-09-01,...,-0.568065,9,-0.866025,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
2,11,0,2,0.000,0,2021-09-01,0,88,44,2021-09-01,...,-0.568065,9,-0.866025,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
3,11,0,1,21.099,1,2021-09-01,0,87,43,2021-09-01,...,-0.568065,9,-0.866025,-0.5,NaN,NaN,NaN,NaN,NaN,NaN
4,11,0,1,0.000,0,2021-09-01,0,86,43,2021-09-01,...,-0.568065,9,-0.866025,-0.5,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time']

X = df_train.drop(['target', 'data_block_id'] + not_feature_columns, axis=1)
y = df_train['target']

# unique year-month combinations -> to be used in cross-validation
timesteps = np.sort(np.array(
    pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
))

## Mlflow + Optuna 
#### Hyperparameter Tuning & Experiment Tracking

In [17]:
from enefit_challenge.models.forecaster import Forecaster

In [28]:
class CatBoostForecaster(Forecaster):
    def __init__(self)-> None:
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        categorical_features: list=[],
        params:Optional[Dict]=None,
        experiment_name: str="catboost",
        artifact_path: str="catboost_model",
        metrics: list=["mae"]
    ) -> CatBoostRegressor:
        """
        Trains a `CatBoostRegressor` with a L1 loss and tracks it using mlflow
        """
        mlflow.set_experiment(experiment_name=experiment_name)
        with mlflow.start_run(nested=True): # mlflow context manager
            model = CatBoostRegressor(
                n_estimators=100, 
                objective='MAE',
                thread_count=1,
                bootstrap_type =  "Bernoulli",
                sampling_frequency= 'PerTree',
                verbose=0,
                cat_features=categorical_features,
                leaf_estimation_iterations=1
            )
            if params:
                model.set_params(**params)

            model.fit(X, y)

            mlflow.catboost.log_model(
                model, 
                artifact_path=artifact_path,
            )
            mlflow.log_params(params)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        categorical_features: list=[],
        experiment_name: str="catboost",
        artifact_path: str="catboost_model",
        metrics: list=["mae"]
    ) -> float:
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)
        X_train = X[train_index];X_test = X[test_index]
        y_train = y[train_index]; y_test = y[test_index]
        # fit model on training data
        model = self.fit_model(
            X_train, 
            y_train, 
            categorical_features,
            params
        )
        # generate predictions
        y_test_pred = model.predict(X_test)
        # signature = infer_signature(X_train, y_test_pred)
        
        return mean_absolute_error(y_test, y_test_pred)

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="catboost",
        artifact_path: str="catboost_model",
        # params: Optional[Dict]=None,
        metrics: list=["mae"]
    ) -> None:
        """ 
        Trains an instance of `CatBoostRegressor` model and tracks the hyperparameter tuning
        experiment with `mlflow` and `optuna`
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list`
            list of categorical features in the dataset
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `metrics`: `list`
            list of the metrics to track in the mlflow experiment run.
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """

        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))

        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name="MAE - Enefit",
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.95,log=True),
                'depth': trial.suggest_int('depth', 3, 10, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8,100,log=True),
                'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1),
                'subsample': trial.suggest_float("subsample", 0.5, 1)
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index],
                    categorical_features
                )
            trial.set_user_attr('split_mae', cv_mae)
            return np.mean(cv_mae)

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=42
        )

        study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name='catboost'
        )

        study.optimize(objective, n_trials=2, timeout= 3600, callbacks=[mlflc]) 

    def forecast():
        pass

In [29]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time']
cat_columns = ['county', 'product_type']

cbf = CatBoostForecaster()
cbf.train_model(
    train_df=df_train,
    target_col="target",
    exclude_cols=not_feature_columns,
    categorical_features=cat_columns,
    )

/var/folders/2c/394jj2r140x4jgq0vzwn32680000gn/T/ipykernel_11780/519825679.py:118: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
/var/folders/2c/394jj2r140x4jgq0vzwn32680000gn/T/ipykernel_11780/519825679.py:122: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow() # decorator to allow mlflow logging
[I 2023-11-12 18:42:58,269] A new study created in memory with name: catboost
/Users/dylantartarini/miniconda3/envs/enefit/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable insta

## Old

In [13]:
def fit_model(
    X:pd.DataFrame,
    y:pd.Series,
    config:Optional[Dict]=None,
    n_jobs:int=1,
    verbose:int=0
) -> CatBoostRegressor:
    """
    Trains a `CatBoostRegressor` with a L1 loss
    """
    model = CatBoostRegressor(
        n_estimators=100, 
        objective='MAE',
        thread_count=n_jobs,
        bootstrap_type =  "Bernoulli",
        sampling_frequency= 'PerTree',
        verbose=verbose,
        cat_features=['county', 'product_type'],
        leaf_estimation_iterations=1
    )
    
    if config:
        # if config is supplied, set the model hyperparameters
        model.set_params(**config)
        
    
    return model.fit(X,y)

def fit_and_test_fold(params:Dict, X, y, year_month_train, year_month_test) -> float:
    first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
    
    train_index = first_dates_month.isin(year_month_train)
    test_index = first_dates_month.isin(year_month_test)
    
    X_train = X[train_index];X_test = X[test_index]
    y_train = y[train_index]; y_test = y[test_index]
    
    # fit model on training data
    model = fit_model(X_train, y_train, params, n_jobs=4)
    
    # generate predictions
    y_test_pred = model.predict(X_test)
    
    return mean_absolute_error(y_test, y_test_pred)

In [14]:
# define mlflow callback Handler for optuna 
mlflc = MLflowCallback(
    metric_name="MAE - Enefit",
)

# loss function
@mlflc.track_in_mlflow() # decorator to allow mlflow logging
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.95,log=True),
        'depth': trial.suggest_int('depth', 3, 10, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8,100,log=True),
        'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
        'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1),
        'subsample': trial.suggest_float("subsample", 0.5, 1)
    }
    cv = TimeSeriesSplit(n_splits=3) # cross validation
    cv_mae = [None]*3
    for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
        cv_mae[i] = fit_and_test_fold(
            params, 
            X, 
            y, 
            timesteps[train_index], 
            timesteps[test_index]
        )
    trial.set_user_attr('split_mae', cv_mae)
        
    return np.mean(cv_mae)

/var/folders/2c/394jj2r140x4jgq0vzwn32680000gn/T/ipykernel_11780/2047370120.py:2: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
/var/folders/2c/394jj2r140x4jgq0vzwn32680000gn/T/ipykernel_11780/2047370120.py:7: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow()


In [16]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials=10, seed=1234
)

study = optuna.create_study(
    directions=['minimize'],
    sampler=sampler,
    study_name='catboost'
)

study.optimize(objective, n_trials=10, timeout= 3600, callbacks=[mlflc]) 

[I 2023-11-12 16:22:04,586] A new study created in memory with name: catboost
2023/11/12 16:22:04 INFO mlflow.tracking.fluent: Experiment with name 'catboost' does not exist. Creating a new experiment.
[I 2023-11-12 16:22:33,028] Trial 0 finished with value: 101.25185644797455 and parameters: {'n_estimators': 78, 'learning_rate': 0.16996488374291047, 'depth': 5, 'l2_leaf_reg': 0.7138152748525242, 'model_size_reg': 0.6306059747250784, 'colsample_bylevel': 0.3453333447543775, 'subsample': 0.6382321275715483}. Best is trial 0 with value: 101.25185644797455.
[I 2023-11-12 16:23:56,877] Trial 1 finished with value: 116.92957519377278 and parameters: {'n_estimators': 171, 'learning_rate': 0.7851177877907137, 'depth': 9, 'l2_leaf_reg': 3.785931044085381e-05, 'model_size_reg': 0.001023178145374356, 'colsample_bylevel': 0.7151166416549227, 'subsample': 0.85635101349145}. Best is trial 0 with value: 101.25185644797455.
[I 2023-11-12 16:24:38,774] Trial 2 finished with value: 90.94295828509344 an